In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-19"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
15001,2024-03-19,Noruega Blno,14:30,Kongsberg Miners,Bærum,1.11,5.77,164.5,1.81,1.85,0.0,0.00,0.00,IsCUWUgr,0.900901,0.173310,0.552486,0.540541,0.074211,118.626,20.878500,0.176003,2.4,1.341641,0.559017,112.53,1.210,0.168671,0.139398,102.0,182.458,143.317902,0.785484,2.4,1.341641,0.559017,130.34,2.266,1.890352,0.834224,33.0,93,98,1.21,1.33,124.956,140.750,0.957883,0.015456,NaN,0.17,0.034,3.235294,0.742626,0.7,-0.042626,-0.86,-0.172,-27.732558,0.506205,0.5,-0.006205
15002,2024-03-19,Israel Liga Leumit,14:30,Ramat Hasharon,Safed,1.75,1.98,161.5,1.85,1.85,-2.5,2.01,2.39,88nwGC2R,0.571429,0.505051,0.540541,0.540541,0.076479,223.558,44.640895,0.199684,0.6,1.341641,2.236068,177.10,2.914,0.566683,0.194469,-53.0,160.174,45.887887,0.286488,1.2,1.643168,1.369306,182.75,2.106,0.583335,0.276987,-19.0,70,85,2.53,2.15,161.376,214.008,0.087204,0.000000,0.122137,-2.55,-0.510,-1.470588,0.000000,0.0,0.000000,-1.98,-0.396,-2.474747,0.000000,0.0,0.000000
15003,2024-03-19,Lituânia Nkl,14:00,BC Olimpas Palanga,Delikatesas Joniskis,1.10,6.15,160.5,1.84,1.86,0.0,0.00,0.00,zyfHppMa,0.909091,0.162602,0.543478,0.537634,0.071693,109.434,47.147027,0.430826,1.8,1.643168,0.912871,109.61,1.406,0.599191,0.426167,-19.0,343.570,241.883876,0.704031,1.8,1.643168,0.912871,470.81,3.950,2.368291,0.599567,-1.0,97,89,1.13,5.29,93.684,284.128,0.985073,0.007644,NaN,0.06,0.012,8.333333,0.818109,0.9,0.081891,7.55,1.510,3.410596,0.398851,0.5,0.101149
15004,2024-03-19,Lituânia Nkl,13:00,Telsiai,Kretinga,1.08,6.69,157.5,1.80,1.86,0.0,0.00,0.00,Gdx6qq7M,0.925926,0.149477,0.555556,0.537634,0.075403,110.628,18.052639,0.163183,1.8,1.643168,0.912871,91.53,1.316,0.254716,0.193553,40.0,467.988,292.155655,0.624280,2.4,1.341641,0.559017,452.35,5.434,2.543134,0.468004,1.0,81,83,1.13,5.45,105.596,436.488,1.021073,0.023184,NaN,-0.73,-0.146,-0.547945,0.744058,0.7,-0.044058,16.28,3.256,1.747543,0.326097,0.6,0.273903
15005,2024-03-19,Japão B2.League,07:00,Shiga,Iwate,1.17,4.56,155.5,1.82,1.86,0.0,0.00,0.00,8CfFrDUa,0.854701,0.219298,0.549451,0.537634,0.073999,109.846,17.367495,0.158108,2.4,1.341641,0.559017,123.20,1.350,0.250500,0.185555,59.0,238.752,88.512453,0.370730,1.2,1.643168,1.369306,168.00,3.104,1.462371,0.471125,-6.0,77,80,1.60,2.10,111.738,331.214,0.836681,0.015372,NaN,1.71,0.342,0.497076,0.774650,0.8,0.025350,9.60,1.920,1.854167,0.287094,0.4,0.112906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15076,2024-03-19,China Cba,08:35,Zhejiang Guangsha,Fujian,1.09,7.01,206.5,1.83,1.89,0.0,0.00,0.00,88nvohZp,0.917431,0.142653,0.546448,0.529101,0.060085,161.954,34.668053,0.214061,1.8,1.643168,0.912871,156.22,1.534,0.297120,0.193689,31.0,266.154,80.907322,0.303987,0.6,1.341641,2.236068,261.32,2.450,0.787496,0.321427,0.0,107,94,1.46,2.78,156.206,409.692,1.033598,0.022810,NaN,-0.34,-0.068,-1.323529,0.739722,0.8,0.060278,-3.41,-0.682,-8.812317,0.337797,0.3,-0.037797
15077,2024-03-19,China Cba,08:35,Tianjin,Beijing Royal Fighters,2.27,1.60,217.5,1.87,1.87,0.0,0.00,0.00,vXt3gGR4,0.440529,0.625000,0.534759,0.534759,0.065529,262.100,88.815743,0.338862,1.2,1.643168,1.369306,354.24,2.540,0

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
15030,22:00,Eua Nit,Butler,Minnesota,1.61,Back Home
15045,15:00,Europa Euroliga,Zalgiris Kaunas,Virtus Bologna,1.63,Back Home
15047,22:00,Eua Nba,Minnesota Timberwolves,Denver Nuggets,2.26,Back Home
15062,16:30,Europa Liga Dos Campeões,Strasbourg,Tofas,1.88,Back Home
15070,20:00,Eua Nit,LSU,North Texas,1.69,Back Home
15073,19:40,Eua Ncaa,Howard,Wagner,1.63,Back Home
15079,11:00,Europa Liga Aba 2,Cedevita,Podgorica,2.50,Back Home
